# **Configurando**
Realizando importações das bibliotecas utilizadas durante a análise exploratória

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import seaborn as sns
import matplotlib.pyplot as plt
import re

Lendo Datasets de Movies e Ratings

In [ ]:
df_movies = pd.read_csv("../../data/movies/raw/movies.csv")
df_ratings = pd.read_csv("../../data/movies/raw/ratings.csv")

In [ ]:
print("Movies Dataset:\n")
print(df_movies.head())

print("Ratings Dataset:\n")
print(df_ratings)


In [ ]:
len(df_movies)

# **Estruturando em um único Dataset**
Adicionado ao dataset Movies as features averageRating e numVotes

In [ ]:
Dataset = pd.merge(df_movies, df_ratings[['tconst', 'averageRating', 'numVotes']], on='tconst', how='left')
print(Dataset.head())
# Quantidade de NaN por coluna
nan_count = Dataset.isna().sum()
print(nan_count)


In [ ]:
len(Dataset)

# **Explorando Features do Dataset**

Vendo os tipos de cada feature

In [ ]:
print(Dataset.dtypes)

Eliminando tconst por se tratar apenas de id do IMDB

In [ ]:
Dataset.drop('tconst', axis=1, inplace=True)
print(Dataset)

Analisando o que tem em cada coluna

In [ ]:
for col in Dataset.columns:
    print(f"\nColuna: {col}")
    print(Dataset[col].unique())


Subdividir a feature categorica 'titleType' em várias features binarias atráves do One Hoting Encode

In [ ]:
Dataset = pd.get_dummies(Dataset, columns=['titleType'], prefix='', prefix_sep='')
print(Dataset.dtypes)

Checando conteúdo dos titulos

In [ ]:
total = len(Dataset)
dif = Dataset['primaryTitle'] != Dataset['originalTitle']
dif_count = dif.sum()
print(f"Diferentes: {dif_count} de {total} ({dif_count/total:.2%})")
diferentes = Dataset[dif]
print(diferentes[['primaryTitle', 'originalTitle']])


São só nomes populares e os nomes originais do filme, pode ser retirado.
Então vamos retirar a coluna 'primaryTitle'

In [ ]:
Dataset.drop('primaryTitle', axis=1, inplace=True)
print(Dataset.head())

Usando One-Hot Encoding para transformar os generos em features boleanas

In [ ]:
Dataset['genres'] = Dataset['genres'].fillna('\\N')

# Transforma em listas (quem for \N vira lista vazia)
Dataset['genres'] = Dataset['genres'].apply(lambda x: [] if x == '\\N' else x.split(','))

# Aplica o MultiLabelBinarizer
mlb = MultiLabelBinarizer()
generos_bin = pd.DataFrame(
    mlb.fit_transform(Dataset['genres']),
    columns=mlb.classes_,
    index=Dataset.index
).astype(bool)

# Junta e remove a coluna original
Dataset = pd.concat([Dataset.drop(columns=['genres']), generos_bin], axis=1)

print(Dataset.head())
print(f"\nTotal de linhas: {len(Dataset)}")

In [ ]:
print(Dataset.dtypes)

In [ ]:
print(Dataset['startYear'].unique())

In [ ]:
Dataset['startYear'] = Dataset['startYear'].replace('\\N', pd.NA)
Dataset['startYear'] = Dataset['startYear'].astype('Int64')  # permite NaN

# 2️⃣ Define intervalos de 5 anos
min_ano = Dataset['startYear'].min(skipna=True)
max_ano = Dataset['startYear'].max(skipna=True)
bins = list(range(min_ano - min_ano % 5, max_ano + 5, 5))
labels = [f'{b}-{b+4}' for b in bins[:-1]]

# 3️⃣ Cria nova coluna com o intervalo de 5 anos
Dataset['yearInterval'] = pd.cut(Dataset['startYear'], bins=bins, labels=labels, right=True)

# 4️⃣ Substitui NaN por 'YearUnknown'
Dataset['yearInterval'] = Dataset['yearInterval'].cat.add_categories(['YearUnknown'])
Dataset['yearInterval'] = Dataset['yearInterval'].fillna('YearUnknown')

# 5️⃣ One-hot encoding
year_dummies = pd.get_dummies(Dataset['yearInterval'], prefix='Year')

# 6️⃣ Junta ao dataset e remove coluna original
Dataset = pd.concat([Dataset.drop(columns=['yearInterval']), year_dummies], axis=1)
Dataset = pd.concat([Dataset.drop(columns=['startYear']), year_dummies], axis=1)

# 7️⃣ Resultado
print(Dataset)

EndYear é apenas o ano que terminou a serie de tv como não se aplica a todos os tipos, pode ser descartado

In [ ]:
Dataset.drop('endYear', axis=1, inplace=True)
print(Dataset.head())

Analisando como está cada coluna

In [ ]:
Dataset['isAdult'] = Dataset['isAdult'].apply(lambda x: False if x == 0 else True)
print(Dataset['isAdult'].unique())


Padronzando runtime minutes para int

In [ ]:
Dataset['runtimeMinutes'] = pd.to_numeric(Dataset['runtimeMinutes'], errors='coerce')
Dataset['runtimeMinutes'] = Dataset['runtimeMinutes'].round(3)
print(Dataset['runtimeMinutes'].head())

Colocando os tempo de duração em categoria, para trabalhar mais fácil.

In [ ]:
bins = [0, 60, 120, 180, np.inf]
labels = ['Curto', 'Médio', 'Longo', 'Extenso']
Dataset['runtimeCategory'] = pd.cut(Dataset['runtimeMinutes'], bins=bins, labels=labels)
Dataset.drop('runtimeMinutes', axis=1, inplace=True)

In [ ]:
print(Dataset['runtimeCategory'])


In [ ]:
# Quantidade de NaN por coluna
nan_count = Dataset.isna().sum()
print(nan_count)


In [ ]:
print(Dataset.dtypes.to_string())
print(Dataset.head)

In [ ]:
# --- 1) Separa grupos automaticamente ---
genre_cols = [
    'Action','Adult','Adventure','Animation','Biography','Comedy','Crime','Documentary',
    'Drama','Family','Fantasy','Film-Noir','Game-Show','History','Horror','Music',
    'Musical','Mystery','News','Reality-TV','Romance','Sci-Fi','Short','Sport',
    'Talk-Show','Thriller','War','Western'
]

type_cols = [
    'movie', 'short', 'tvEpisode', 'tvMiniSeries', 'tvMovie', 'tvPilot',
    'tvSeries', 'tvShort', 'tvSpecial', 'video', 'videoGame'
]

year_cols = [col for col in Dataset.columns if col.startswith("Year_")]

# --- 2) Soma os valores True ---
genre_counts = Dataset[genre_cols].sum().sort_values(ascending=False)
type_counts  = Dataset[type_cols].sum().sort_values(ascending=False)
year_counts  = Dataset[year_cols].sum().sort_values(ascending=False)

# --- 3) Remover valores zero ---
genre_counts = genre_counts[genre_counts > 0]
type_counts  = type_counts[type_counts > 0]
year_counts  = year_counts[year_counts > 0]


# ==============================
#  GRÁFICOS DE GÊNEROS
# ==============================

# Pizza
plt.figure(figsize=(10, 10))
plt.pie(genre_counts, labels=genre_counts.index, autopct='%1.1f%%')
plt.title("Distribuição de Gêneros")
plt.show()

# Barras
plt.figure(figsize=(12, 6))
plt.bar(genre_counts.index, genre_counts)
plt.xticks(rotation=90)
plt.title("Contagem por Gênero")
plt.xlabel("Gênero")
plt.ylabel("Quantidade")
plt.show()


# ==============================
#  GRÁFICOS DE TIPOS
# ==============================

# Pizza
plt.figure(figsize=(10, 10))
plt.pie(type_counts, labels=type_counts.index, autopct='%1.1f%%')
plt.title("Distribuição de Tipos")
plt.show()

# Barras
plt.figure(figsize=(12, 6))
plt.bar(type_counts.index, type_counts)
plt.xticks(rotation=45)
plt.title("Contagem por Tipo")
plt.xlabel("Tipo")
plt.ylabel("Quantidade")
plt.show()


# ==============================
#  GRÁFICOS DE INTERVALOS DE ANO
# ==============================

# Pizza
plt.figure(figsize=(10, 10))
plt.pie(year_counts, labels=year_counts.index, autopct='%1.1f%%')
plt.title("Distribuição por Intervalo de Ano")
plt.show()

# Barras
plt.figure(figsize=(12, 6))
plt.bar(year_counts.index, year_counts)
plt.xticks(rotation=90)
plt.title("Contagem por Intervalo de Ano")
plt.xlabel("Intervalo de Ano")
plt.ylabel("Quantidade")
plt.show()


Agora que já tenho as metricas, excluo as linhas que não tem nossa celula objetivo pq é irrelevante

In [ ]:
Dataset = Dataset.dropna(subset=['averageRating'])

In [ ]:
len(Dataset)

In [ ]:
# --- 1) Separa grupos automaticamente ---
genre_cols = [
    'Action','Adult','Adventure','Animation','Biography','Comedy','Crime','Documentary',
    'Drama','Family','Fantasy','Film-Noir','Game-Show','History','Horror','Music',
    'Musical','Mystery','News','Reality-TV','Romance','Sci-Fi','Short','Sport',
    'Talk-Show','Thriller','War','Western'
]

type_cols = [
    'movie', 'short', 'tvEpisode', 'tvMiniSeries', 'tvMovie', 'tvPilot',
    'tvSeries', 'tvShort', 'tvSpecial', 'video', 'videoGame'
]

year_cols = [col for col in Dataset.columns if col.startswith("Year_")]

# --- 2) Soma os valores True ---
genre_counts = Dataset[genre_cols].sum().sort_values(ascending=False)
type_counts  = Dataset[type_cols].sum().sort_values(ascending=False)
year_counts  = Dataset[year_cols].sum().sort_values(ascending=False)

# --- 3) Remover valores zero ---
genre_counts = genre_counts[genre_counts > 0]
type_counts  = type_counts[type_counts > 0]
year_counts  = year_counts[year_counts > 0]


# ==============================
#  GRÁFICOS DE GÊNEROS
# ==============================

# Pizza
plt.figure(figsize=(10, 10))
plt.pie(genre_counts, labels=genre_counts.index, autopct='%1.1f%%')
plt.title("Distribuição de Gêneros")
plt.show()

# Barras
plt.figure(figsize=(12, 6))
plt.bar(genre_counts.index, genre_counts)
plt.xticks(rotation=90)
plt.title("Contagem por Gênero")
plt.xlabel("Gênero")
plt.ylabel("Quantidade")
plt.show()


# ==============================
#  GRÁFICOS DE TIPOS
# ==============================

# Pizza
plt.figure(figsize=(10, 10))
plt.pie(type_counts, labels=type_counts.index, autopct='%1.1f%%')
plt.title("Distribuição de Tipos")
plt.show()

# Barras
plt.figure(figsize=(12, 6))
plt.bar(type_counts.index, type_counts)
plt.xticks(rotation=45)
plt.title("Contagem por Tipo")
plt.xlabel("Tipo")
plt.ylabel("Quantidade")
plt.show()


# ==============================
#  GRÁFICOS DE INTERVALOS DE ANO
# ==============================

# Pizza
plt.figure(figsize=(10, 10))
plt.pie(year_counts, labels=year_counts.index, autopct='%1.1f%%')
plt.title("Distribuição por Intervalo de Ano")
plt.show()

# Barras
plt.figure(figsize=(12, 6))
plt.bar(year_counts.index, year_counts)
plt.xticks(rotation=90)
plt.title("Contagem por Intervalo de Ano")
plt.xlabel("Intervalo de Ano")
plt.ylabel("Quantidade")
plt.show()


In [ ]:
genre_cols = [
    'Action','Adult','Adventure','Animation','Biography','Comedy','Crime','Documentary',
    'Drama','Family','Fantasy','Film-Noir','Game-Show','History','Horror','Music',
    'Musical','Mystery','News','Reality-TV','Romance','Sci-Fi','Short','Sport',
    'Talk-Show','Thriller','War','Western'
]

type_cols = [
    'movie', 'short', 'tvEpisode', 'tvMiniSeries', 'tvMovie', 'tvPilot',
    'tvSeries', 'tvShort', 'tvSpecial', 'video', 'videoGame'
]

year_cols = [col for col in Dataset.columns if col.startswith("Year_")]

# --- 2) Soma dos valores True ---
genre_counts = Dataset[genre_cols].sum().sort_values(ascending=False)
type_counts  = Dataset[type_cols].sum().sort_values(ascending=False)
year_counts  = Dataset[year_cols].sum().sort_values(ascending=False)

# --- 3) Remove zeros ---
genre_counts = genre_counts[genre_counts > 0]
type_counts  = type_counts[type_counts > 0]
year_counts  = year_counts[year_counts > 0]


# ===========================================
# FUNÇÕES AUXILIARES PARA MELHORAR A QUALIDADE
# ===========================================

def plot_pie(values, title):
    plt.figure(figsize=(10, 10))
    explode = [0.07 if v == values.max() else 0 for v in values]  # destaca maior slice
    plt.pie(values, labels=values.index, autopct='%1.1f%%', pctdistance=0.85, explode=explode)
    plt.title(title, fontsize=16)
    centre_circle = plt.Circle((0,0),0.60,fc='white')  # donut
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    plt.tight_layout()
    plt.show()

def plot_bar(values, title, xlabel):
    plt.figure(figsize=(12, 6))
    bars = plt.bar(values.index, values)

    # Adiciona número nas barras
    for bar in bars:
        y = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, y, f'{int(y)}',
                 ha='center', va='bottom', fontsize=9)

    plt.title(title, fontsize=15)
    plt.xlabel(xlabel)
    plt.ylabel("Quantidade")
    plt.xticks(rotation=90)
    plt.grid(axis='y', linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.show()

def plot_barh(values, title, ylabel):
    plt.figure(figsize=(10, 8))
    bars = plt.barh(values.index, values)

    # Adiciona valores à direita das barras
    for bar in bars:
        x = bar.get_width()
        plt.text(x + x*0.01, bar.get_y() + bar.get_height()/2,
                 f'{int(x)}', va='center')

    plt.title(title, fontsize=15)
    plt.ylabel(ylabel)
    plt.xlabel("Quantidade")
    plt.grid(axis='x', linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.show()


# ==============================
#  GRÁFICOS MELHORADOS
# ==============================

# --- Gêneros ---
plot_pie(genre_counts, "Distribuição de Gêneros (Somente valores > 0)")
plot_barh(genre_counts, "Contagem por Gênero", "Gênero")

# --- Tipos ---
plot_pie(type_counts, "Distribuição de Tipos (Somente valores > 0)")
plot_bar(type_counts, "Contagem por Tipo", "Tipo")

# --- Anos ---
plot_pie(year_counts, "Distribuição por Intervalo de Ano (Somente valores > 0)")
plot_bar(year_counts, "Contagem por Intervalo de Ano", "Intervalo de Ano")

In [ ]:
# --- Seleciona apenas colunas de anos ---
year_cols = [col for col in Dataset.columns if col.startswith("Year_")]

# --- Soma True por coluna ---
year_counts = Dataset[year_cols].sum()

# --- Remove anos com zero ocorrências ---
year_counts = year_counts[year_counts > 0]

# --- Extrai o ano inicial do intervalo e ordena ---
def extract_year(col):
    # Exemplo: Year_1990-1994 → 1990
    match = re.search(r'Year_(\d+)', col)
    return int(match.group(1)) if match else 999999

sorted_years = year_counts.index.tolist()
sorted_years = sorted(sorted_years, key=extract_year)

year_counts = year_counts[sorted_years]

# --- Plot em ordem cronológica ---
plt.figure(figsize=(14, 6))
bars = plt.bar(year_counts.index, year_counts)

# Adiciona valores no topo das barras
for bar in bars:
    y = bar.get_height()
    offset = max(year_counts.values)*0.01
    plt.text(
        bar.get_x() + bar.get_width()/2,
        y + offset,
        str(int(y)),  # Adicionado o argumento 's' para o texto
        ha='center',
        fontsize=10,
        rotation=90  # opcional: deixa vertical para barras muito próximas
    )

plt.title("Contagem por Intervalo de Ano (Ordem Cronológica)")
plt.xlabel("Intervalos de Ano")
plt.ylabel("Quantidade")
plt.xticks(rotation=45)

plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate average rating per genre
genre_avg_ratings = {}
for genre in genre_cols:
    genre_avg_ratings[genre] = Dataset[Dataset[genre] == True]['averageRating'].mean()

genre_avg_ratings_series = pd.Series(genre_avg_ratings).sort_values(ascending=False)

# Calculate average rating per type
type_avg_ratings = {}
for movie_type in type_cols:
    type_avg_ratings[movie_type] = Dataset[Dataset[movie_type] == True]['averageRating'].mean()

type_avg_ratings_series = pd.Series(type_avg_ratings).sort_values(ascending=False)

# Plotting average ratings for genres
plt.figure(figsize=(14, 7))
sns.barplot(x=genre_avg_ratings_series.index, y=genre_avg_ratings_series.values, palette='viridis')
plt.title('Média de Avaliação por Gênero')
plt.xlabel('Gênero')
plt.ylabel('Média de Avaliação')
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Plotting average ratings for types
plt.figure(figsize=(14, 7))
sns.barplot(x=type_avg_ratings_series.index, y=type_avg_ratings_series.values, palette='plasma')
plt.title('Média de Avaliação por Tipo de Conteúdo')
plt.xlabel('Tipo de Conteúdo')
plt.ylabel('Média de Avaliação')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Aplica One-Hot Encoding na coluna 'runtimeCategory'
runtime_dummies = pd.get_dummies(Dataset['runtimeCategory'], prefix='runtimeCategory')

# Concatena as novas colunas ao Dataset
Dataset = pd.concat([Dataset, runtime_dummies], axis=1)

# Remove a coluna original 'runtimeCategory'
Dataset.drop('runtimeCategory', axis=1, inplace=True)

# Calcula as contagens para as novas colunas de categoria de tempo de execução
runtime_category_counts = runtime_dummies.sum().sort_values(ascending=False)

# Plota o gráfico de pizza
plt.figure(figsize=(10, 8))
plt.pie(runtime_category_counts, labels=runtime_category_counts.index, autopct='%1.1f%%', startangle=90)
plt.title(
    'Distribuição de Filmes por Categoria de Duração\n'
    'Curto: 0-60 min | Médio: 60-120 min | Longo: 120-180 min | Extenso: >180 min',
    fontsize=12
)
plt.axis('equal') # Garante que o gráfico de pizza seja circular.
plt.show()

As categorias de duração foram definidas da seguinte forma:

* **Curto**: Filmes com duração entre 0 e 60 minutos.
* **Médio**: Filmes com duração entre 60 e 120 minutos.
* **Longo**: Filmes com duração entre 120 e 180 minutos.
* **Extenso**: Filmes com duração acima de 180 minutos.

In [ ]:
plt.figure(figsize=(10, 8))
plt.pie(
    runtime_category_counts,
    labels=runtime_category_counts.index.str.replace('runtimeCategory_', ''),
    autopct='%1.1f%%',
    startangle=90,
)

# Título
plt.title(
    'Distribuição de Filmes por Categoria de Duração',
    fontsize=14,
    fontweight='bold'
)

# Caixa de informação no canto
info_text = (
    "Categorias de duração:\n"
    "Curto: 0 – 60 min\n"
    "Médio: 60 – 120 min\n"
    "Longo: 120 – 180 min\n"
    "Extenso: > 180 min"
)

plt.text(
    1.1, 0.5, info_text,
    fontsize=10,
    verticalalignment='bottom',
    bbox=dict(boxstyle="round,pad=0.5", fc="#f0f0f0", ec="gray", alpha=0.8)
)

plt.axis('equal')
plt.tight_layout()
plt.show()